In [ ]:
# Tüm adımlar için ortak
# 1. HÜCRE: Kütüphaneler ve Veri Hazırlığı
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Veriyi Yükle
df = pd.read_parquet("target2_data.parquet")

# X ve y ayır
X = df.drop(columns=['target'])
y = df['target']

# Eğitim ve Test seti (%80 - %20)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# Lojistik Regresyon için SCALING (Ölçeklendirme) ŞARTTIR
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Veri yüklendi ve ölçeklendirildi.")
print(f"Eğitim Seti Boyutu: {X_train_scaled.shape}")

✅ Veri yüklendi ve ölçeklendirildi.
Eğitim Seti Boyutu: (49145, 41)


In [ ]:
# 2. HÜCRE: Naive (Varsayılan) Model Eğitimi

print("1. Aşama: Varsayılan (Naive) Model")

# Hiçbir parametre vermiyoruz (max_iter hariç, o da hata vermemesi için)
naive_model = LogisticRegression(max_iter=1000, random_state=42)

# Modeli eğit
naive_model.fit(X_train_scaled, y_train)

# Tahmin yap
y_pred_naive = naive_model.predict(X_test_scaled)
acc_naive = accuracy_score(y_test, y_pred_naive)

print(f"\n--- NAIVE MODEL SONUÇLARI ---")
print(f"Doğruluk (Accuracy): %{acc_naive*100:.2f}")
print("-" * 30)
print(classification_report(y_test, y_pred_naive))

1. Aşama: Varsayılan (Naive) Model Eğitiliyor...

--- NAIVE MODEL SONUÇLARI ---
Doğruluk (Accuracy): %83.17
------------------------------
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      9020
           1       0.53      0.04      0.07      1551
           2       0.69      0.78      0.73      1716

    accuracy                           0.83     12287
   macro avg       0.69      0.60      0.57     12287
weighted avg       0.80      0.83      0.78     12287



### Sonuç

"Naive (Varsayılan) modelimiz %83.17 genel doğruluk oranına ulaşsa da, bu skor yanıltıcıdır. Sınıflandırma raporunu incelediğimizde, modelin 'Orta Popülerlik (Class 1)' sınıfını tahmin etmekte başarısız olduğu (Recall: 0.04) görülmüştür. Model, veri setindeki dengesizlikten dolayı ağırlıklı olarak çoğunluk sınıfı olan 'Düşük Popülerlik' sınıfına odaklanmıştır."

Şimdi Ne Yapacağız?

Bir sonraki adımda yapacağımız Hiperparametre Optimizasyonu (GridSearch) sırasında, modelin bu dengesizliği çözmesi için ona "class_weight='balanced'" (Sınıfları dengele) ayarını da eklememiz çok iyi olur.

Devam edip optimizasyon kodunu çalıştırdığımızda, muhtemelen genel doğruluk (accuracy) biraz düşebilir ama Class 1 (Orta) sınıfını yakalama oranı (Recall) artacaktır. Bu da daha sağlıklı bir model demektir.

1. En Kritik Parametreler (Mutlaka Bilmen Gerekenler)

Bu parametreler modelin başarısını doğrudan etkiler. GridSearch yaparken en çok bunlarla oynarız.

penalty (Ceza / Düzenlileştirme Türü)

Ne Yapar? Modelin ezberlemesini (overfitting) engellemek için kullanılan "fren" mekanizmasıdır. Gereksiz veya gürültülü özelliklerin etkisini kısar.

Seçenekler:

'l2': (Varsayılan) Katsayıların karesini alarak cezalandırır. Tüm özelliklerin etkisini biraz azaltır ama sıfırlamaz. Genelde en güvenli limandır.

'l1': Katsayıların mutlak değerini alır. Gereksiz gördüğü özelliklerin katsayısını tamamen sıfırlar. Yani özellik seçimi (feature selection) yapar.

'elasticnet': L1 ve L2'nin karışımıdır.

None: Ceza yok, model tamamen serbest. (Ezberlemeye çok açık olur).

C (Düzenlileştirme Gücünün Tersi)

Ne Yapar? Cezanın şiddetini ayarlar. Ama dikkat: Ters orantılıdır.

Küçük C (örn: 0.01, 0.1): Güçlü ceza. Model çok "basit" kalmaya zorlanır. Hata yapmayı göze alır ama genelleme yeteneği artar (Underfitting riski).

Büyük C (örn: 10, 100): Zayıf ceza. Model veriye sıkı sıkıya uymaya çalışır. Eğitim setinde çok başarılı olur ama yeni veride çuvallayabilir (Overfitting riski).

Özet: C küçüldükçe model "muhafazakar", büyüdükçe "agresif" olur.

solver (Çözücü / Matematik Motoru)

Ne Yapar? "En iyi katsayıları bulmak için hangi matematiksel algoritmayı kullanayım?" sorusunun cevabıdır.

Seçenekler:

'liblinear': Küçük veri setleri için harikadır. Sadece 'l1' ve 'l2' destekler.

'lbfgs': (Varsayılan) Çoğu durumda en iyisi. Çok sınıflı (multinomial) problemlerde güçlüdür.

'saga': Çok büyük veri setlerinde hızlıdır. 'elasticnet' destekleyen tek çözücüdür.

'newton-cg', 'sag': Genelde büyük verilerde L2 cezası ile kullanılır.

class_weight (Sınıf Ağırlığı)

Ne Yapar? Senin projen için en hayati parametrelerden biriydi!

Seçenekler:

None: (Varsayılan) Her sınıfa eşit muamele yapar. (Senin ilk denemende "Orta" sınıfı ezmesinin sebebi buydu).

'balanced': Az bulunan sınıfa (Orta Popülerlik) daha fazla önem verir, çok bulunan sınıfa (Düşük Popülerlik) daha az önem verir. Hatayı buna göre cezalandırır.

dict: Elle ayar verebilirsin (Örn: {0:1, 1:10, 2:1}).

2. İnce Ayar Parametreleri (Konfigürasyon)

Bu parametreler modelin çalışma şeklini ve hızını etkiler.

max_iter (Maksimum İterasyon)

Ne Yapar? Modelin çözümü bulmak için deneme sayısı limiti.

Varsayılan: 100.

Ne zaman değiştirmeli? Eğer "ConvergenceWarning" (Yakınsamadı hatası) alırsan bu sayıyı artır (örn: 1000 veya 2000 yap). Veri karmaşıksa modelin ders çalışmak için daha fazla süreye ihtiyacı vardır.

tol (Tolerans / Durma Kriteri)

Ne Yapar? Model ne zaman "Tamam, bulduğum sonuç yeterince iyi, artık durayım" desin?

Detay: Hata değişimi bu sayıdan (örn: 0.0001) küçükse eğitimi durdurur. Çok küçültürsen eğitim uzar, çok büyütürsen model tam öğrenmeden durur.

fit_intercept (Kesişim Noktası)

Ne Yapar? Doğrunun başlangıç noktasını (bias / w 
0
​	
 ) hesaplasın mı?

Detay: Genelde True kalır. Verinin merkezini (0,0) noktasından geçmeye zorlamak istersen False yaparsın (nadiren gerekir).

multi_class (Çoklu Sınıf Yöntemi)

Ne Yapar? Hedef değişkenin 2'den fazla sınıfı varsa (Seninkinde 3 tane: Low, Medium, High) nasıl davranacağını seçer.

Seçenekler:

'auto': (Varsayılan) Veriye bakıp kendi seçer.

'ovr' (One-vs-Rest): Her sınıf için ayrı ayrı "Bu mu, değil mi?" modelleri kurar.

'multinomial': Tüm sınıfları tek bir matematiksel formülde (Softmax) çözer. Genelde daha doğrudur.

random_state (Rastgelelik Tohumu)

Ne Yapar? Sonuçların her çalıştırdığında aynı çıkmasını sağlar. Bilimsel çalışmalarda sabitlemek (örn: 42) önemlidir.

n_jobs (İşlemci Gücü)

Ne Yapar? Bilgisayarın kaç çekirdeğini kullanacağını seçer.

Ayar: -1 yaparsan bilgisayarın tüm gücünü kullanır ve işlem hızlanır.

3. Nadir Kullanılanlar (Meraklısına)

dual (Primal vs Dual)

Ne Yapar? Matematiksel çözüm yöntemidir.

Kural: Eğer satır sayısı (n_samples), sütun sayısından (n_features) fazlaysa dual=False (Varsayılan) olmalıdır. Senin verinde satır çok olduğu için False kalmalı.

l1_ratio (L1 Oranı)

Ne Yapar? Sadece penalty='elasticnet' seçersen çalışır.

Detay: 0 ile 1 arasında bir sayıdır. 1 olursa tamamen L1 gibi, 0 olursa tamamen L2 gibi davranır. 0.5 ise yarı yarıya karışım yapar.

intercept_scaling

Sadece liblinear kullanıldığında devreye giren, bias teriminin ölçeklendirilmesiyle ilgili çok spesifik bir ayardır. Genelde dokunulmaz.

warm_start (Sıcak Başlangıç)

Ne Yapar? True yaparsan, modeli tekrar eğittiğinde sıfırdan başlamaz, önceki eğitimin kaldığı yerden devam eder. GridSearch gibi süreçlerde genelde False kullanılır.

verbose

Ne Yapar? Eğitim sırasında ekrana ne kadar bilgi basılacağını ayarlar. 0 ise sessizdir, sayı arttıkça log basar.

In [ ]:
# GridSearch ile kombinasyonları deneme
from sklearn.model_selection import GridSearchCV

print("2. Aşama: En İyi Parametreler Aranıyor (GridSearch)")

# Senin belirlediğin stratejik ayarlar
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],        # Modelin esnekliği
    'class_weight': [None, 'balanced'],  # Dengesizlik çözümü (Kritik!)
    'solver': ['lbfgs', 'liblinear'],    # Matematik motoru
    'max_iter': [2000]                   # Hata almamak için iterasyon limiti artırıldı
}

# Modeli başlat
log_model = LogisticRegression(random_state=42)

# GridSearch'ü kur
# cv=5: Her denemeyi 5 farklı veri parçasında test edip sağlamasını yapacak.
grid_search = GridSearchCV(
    estimator=log_model,
    param_grid=param_grid,
    cv=5,   # Cross- validaiton yap
    scoring='accuracy',
    n_jobs=-1,  # Tüm işlemci gücünü kullan
    verbose=1
)

# Aramayı Başlat (Scaled veriyi kullanıyoruz!)
grid_search.fit(X_train_scaled, y_train)

# KAZANAN AYARLARI GÖSTER
print(f"\nEn İyi Parametreler: {grid_search.best_params_}")
print(f"En İyi Eğitim Skoru: %{grid_search.best_score_*100:.2f}")

# En iyi modeli bir değişkene atayalım ki aşağıda kullanalım
optimized_model = grid_search.best_estimator_

2. Aşama: En İyi Parametreler Aranıyor (GridSearch)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/iremkeser/Desktop/anaconda3/envs/steam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Users/iremkeser/Desktop/anaconda3/envs/steam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Users/iremkeser/Desktop/anaconda3/envs/steam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification i


✅ En İyi Parametreler: {'C': 1, 'class_weight': None, 'max_iter': 2000, 'solver': 'lbfgs'}
✅ En İyi Eğitim Skoru: %82.84


In [13]:
# =============================================================================
# 4. HÜCRE: Final Test ve Görselleştirme
# =============================================================================
from sklearn.metrics import confusion_matrix

# 1. Tahmin Yap
y_pred_opt = optimized_model.predict(X_test_scaled)
acc_opt = accuracy_score(y_test, y_pred_opt)

print(f"\n--- OPTİMİZE EDİLMİŞ MODEL (TEST SETİ) ---")
print(f"Doğruluk (Accuracy): %{acc_opt*100:.2f}")
print("-" * 30)
# Bu rapor bize Recall değerlerinin düzelip düzelmediğini söyleyecek
print(classification_report(y_test, y_pred_opt))

# 2. Hata Matrisi (Görsel)
#plt.figure(figsize=(6, 5))
#cm = confusion_matrix(y_test, y_pred_opt)
#sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
#            xticklabels=['Low (0)', 'Medium (1)', 'High (2)'], 
#            yticklabels=['Low (0)', 'Medium (1)', 'High (2)'])
#plt.title('Optimize Model Hata Matrisi')
#plt.ylabel('Gerçek Durum')
#plt.xlabel('Modelin Tahmini')
#plt.show()

# 3. Hangi Özellikler En Önemli Çıktı? (Katsayı Analizi)
# Lojistik Regresyonun en güçlü yanı bu tablodur.
if hasattr(optimized_model, 'coef_'):
    # Class 2 (High - Popüler) için katsayıları alalım
    # Pozitif değerler popülerliği artırır, negatifler azaltır.
    coeffs = optimized_model.coef_[2] 
    
    feature_importance = pd.DataFrame({'Feature': X.columns, 'Coefficient': coeffs})
    feature_importance['Abs_Coeff'] = feature_importance['Coefficient'].abs()
    feature_importance = feature_importance.sort_values(by='Abs_Coeff', ascending=False).head(15)

    #plt.figure(figsize=(10, 6))
    #sns.barplot(x='Coefficient', y='Feature', data=feature_importance, palette='coolwarm')
    #plt.title('Yüksek Popülerliği (High) Etkileyen En Önemli Faktörler')
    #plt.axvline(x=0, color='black', linestyle='--')
    #plt.show()


--- OPTİMİZE EDİLMİŞ MODEL (TEST SETİ) ---
Doğruluk (Accuracy): %83.17
------------------------------
              precision    recall  f1-score   support

           0       0.86      0.98      0.92      9020
           1       0.53      0.04      0.07      1551
           2       0.69      0.78      0.73      1716

    accuracy                           0.83     12287
   macro avg       0.69      0.60      0.57     12287
weighted avg       0.80      0.83      0.78     12287



In [14]:
# =============================================================================
# 3. HÜCRE: GridSearch ile En İyi Ayarları Bulma
# =============================================================================
from sklearn.model_selection import GridSearchCV

print("2. Aşama: En İyi Parametreler Aranıyor (GridSearch)...")

# Senin belirlediğin stratejik ayarlar
param_grid = {
    'C': [0.01, 0.1, 1, 10, 100],        # Modelin esnekliği
    'class_weight': [None, 'balanced'],  # Dengesizlik çözümü (Kritik!)
    'solver': ['lbfgs', 'liblinear'],    # Matematik motoru
    'max_iter': [2000]                   # Hata almamak için iterasyon limiti artırıldı
}

# Modeli başlat
log_model = LogisticRegression(random_state=42)

# GridSearch'ü kur
# cv=5: Her denemeyi 5 farklı veri parçasında test edip sağlamasını yapacak.
grid_search = GridSearchCV(
    estimator=log_model,
    param_grid=param_grid,
    cv=5,
    scoring='f1_macro',
    n_jobs=-1,  # Tüm işlemci gücünü kullan
    verbose=1
)

# Aramayı Başlat (Scaled veriyi kullanıyoruz!)
grid_search.fit(X_train_scaled, y_train)

# KAZANAN AYARLARI GÖSTER
print(f"\n✅ En İyi Parametreler: {grid_search.best_params_}")
print(f"✅ En İyi Eğitim Skoru: %{grid_search.best_score_*100:.2f}")

# En iyi modeli bir değişkene atayalım ki aşağıda kullanalım
optimized_model = grid_search.best_estimator_

2. Aşama: En İyi Parametreler Aranıyor (GridSearch)...
Fitting 5 folds for each of 20 candidates, totalling 100 fits


/Users/iremkeser/Desktop/anaconda3/envs/steam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Users/iremkeser/Desktop/anaconda3/envs/steam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification is deprecated. An error will be raised in 1.8. Either use another solver which supports the multinomial loss or wrap the estimator in a OneVsRestClassifier to keep applying a one-versus-rest scheme.
  warnings.warn(
/Users/iremkeser/Desktop/anaconda3/envs/steam/lib/python3.10/site-packages/sklearn/linear_model/_logistic.py:1296: FutureWarning: Using the 'liblinear' solver for multiclass classification i


✅ En İyi Parametreler: {'C': 0.01, 'class_weight': 'balanced', 'max_iter': 2000, 'solver': 'lbfgs'}
✅ En İyi Eğitim Skoru: %62.33


In [15]:
from sklearn.metrics import confusion_matrix

# 1. Tahmin Yap
y_pred_opt = optimized_model.predict(X_test_scaled)
acc_opt = accuracy_score(y_test, y_pred_opt)

print(f"\n--- OPTİMİZE EDİLMİŞ MODEL (TEST SETİ) ---")
print(f"Doğruluk (Accuracy): %{acc_opt*100:.2f}")
print("-" * 30)
# Bu rapor bize Recall değerlerinin düzelip düzelmediğini söyleyecek
print(classification_report(y_test, y_pred_opt))


--- OPTİMİZE EDİLMİŞ MODEL (TEST SETİ) ---
Doğruluk (Accuracy): %73.92
------------------------------
              precision    recall  f1-score   support

           0       0.92      0.79      0.85      9020
           1       0.25      0.43      0.32      1551
           2       0.68      0.77      0.72      1716

    accuracy                           0.74     12287
   macro avg       0.62      0.66      0.63     12287
weighted avg       0.80      0.74      0.76     12287

